In [1]:
#!pip install openpyxl

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import datetime 

In [2]:
data_path = './data'
out_path = './output'

# BeautifulSoup 활용(여려 페이지 가져오기)

## 1 네이버 금융 -> 국내증시 -> 시가총액

In [16]:
url ='https://finance.naver.com/sise/sise_market_sum.naver'
response = requests.get(url)
soup = bs(response.text,'html.parser')

In [17]:
# 코스피, 코스닥 같은 페이지인데 파라미터로 구분이 된다.
# 코스피 : https://finance.naver.com/sise/sise_market_sum.naver?sosok=0
# 코스닥 : https://finance.naver.com/sise/sise_market_sum.naver?sosok=1
# 페이지 번호도 파라미터로 표시 된다. (예시 1페이지)
# https://finance.naver.com/sise/sise_market_sum.naver?&page=1
# param = {'sosok':1, 'page':3} # 코스닥, 3페이지

In [18]:
thead = soup.select_one('#contentarea > div.box_type_l > table.type_2 > thead')
thead_th = thead.select('th')
print(thead_th)
print(thead_th[2].text)

[<th scope="col">N</th>, <th scope="col">종목명</th>, <th scope="col">현재가</th>, <th class="tr" scope="col" style="padding-right:8px">전일비</th>, <th scope="col">등락률</th>, <th scope="col">액면가</th>, <th scope="col">시가총액</th>, <th scope="col">상장주식수</th>, <th scope="col">외국인비율</th>, <th scope="col">거래량</th>, <th scope="col">PER</th>, <th scope="col">ROE</th>, <th scope="col">토론실</th>]
현재가


In [19]:
for i, th in enumerate(thead_th):
    print(f'{i} 번쨰 : {th.text}')

0 번쨰 : N
1 번쨰 : 종목명
2 번쨰 : 현재가
3 번쨰 : 전일비
4 번쨰 : 등락률
5 번쨰 : 액면가
6 번쨰 : 시가총액
7 번쨰 : 상장주식수
8 번쨰 : 외국인비율
9 번쨰 : 거래량
10 번쨰 : PER
11 번쨰 : ROE
12 번쨰 : 토론실


In [20]:
print(len(thead_th)) # 새로(column)이 13개인 것을 알 수 있다.

13


### 1페이지

In [21]:
rows = soup.select('#contentarea > div.box_type_l > table.type_2 > tbody > tr')
# print(rows)
print(type(rows))
print(len(rows))

<class 'bs4.element.ResultSet'>
81


In [22]:
# for row in rows:
#     tags =row.select('td')
#     print(tags.text)
# find_all() 메서드는 조건에 맞는 모든 요소들을 리스트로 반환합니다. 
# 따라서 find_all()의 결과는 리스트 형태입니다.
# 리스트에는 text 속성이 없기 때문에, 
# 리스트에서 text 속성을 직접 호출하려고 하면 AttributeError가 발생합

In [27]:
title_list = []
for data in rows:
    title = data.select_one('.tltle')
    if title == None:
        pass
    else:
        title_list.append(title.text)
        # print(title.text)
title_list[:10]

['삼성전자삼성전자',
 'SK하이닉스SK하이닉스',
 'LG에너지솔루션LG에너지솔루션',
 '삼성바이오로직스삼성바이오로직스',
 '현대차현대차',
 '셀트리온셀트리온',
 '삼성전자우삼성전자우',
 '기아기아',
 'KB금융KB금융',
 'POSCO홀딩스POSCO홀딩스']

In [28]:
# N, 종목명, 링크, code, 현재가, 시가총액, 거래량
https = 'https://finance.naver.com/'
for row in rows:
    tags = row.select('td') # td(칸)
    # print(tags)
    # print(len(tags)) # td의 개수 -> 13개인것만 갖고 오기
    if len(tags) == 13:
        print(f'N : {tags[0].text}', end = ' ') # end = ' ' 엔터를 공백으로 대체
        print(f'종목명 : {tags[1].text}', end = ' ')
        print(f'링크 : {https}{tags[1].select_one("a").get("href")}', end = ' ')
        print(f'code : {tags[1].select_one("a").get("href")[-6:]}', end = ' ')
        print(f'현재가 : {tags[2].text}', end = ' ')
        print(f'시가총액 : {tags[6].text}', end = ' ')
        print(f'거래량 : {tags[9].text}', end = ' ')
        print()
        print()

N : 1 종목명 : 삼성전자삼성전자 링크 : https://finance.naver.com//item/main.naver?code=005930 code : 005930 현재가 : 59,100 시가총액 : 3,528,141 거래량 : 19,176,141 

N : 2 종목명 : SK하이닉스SK하이닉스 링크 : https://finance.naver.com//item/main.naver?code=000660 code : 000660 현재가 : 195,000 시가총액 : 1,419,605 거래량 : 3,439,443 

N : 3 종목명 : LG에너지솔루션LG에너지솔루션 링크 : https://finance.naver.com//item/main.naver?code=373220 code : 373220 현재가 : 411,500 시가총액 : 962,910 거래량 : 171,053 

N : 4 종목명 : 삼성바이오로직스삼성바이오로직스 링크 : https://finance.naver.com//item/main.naver?code=207940 code : 207940 현재가 : 1,031,000 시가총액 : 733,804 거래량 : 64,569 

N : 5 종목명 : 현대차현대차 링크 : https://finance.naver.com//item/main.naver?code=005380 code : 005380 현재가 : 220,500 시가총액 : 461,763 거래량 : 541,960 

N : 6 종목명 : 셀트리온셀트리온 링크 : https://finance.naver.com//item/main.naver?code=068270 code : 068270 현재가 : 187,700 시가총액 : 407,349 거래량 : 301,457 

N : 7 종목명 : 삼성전자우삼성전자우 링크 : https://finance.naver.com//item/main.naver?code=005935 code : 005935 현재가 : 48,650 시가총액 : 400,334 거래량 : 1,

### 2페이지

In [29]:
url = 'https://finance.naver.com/sise/sise_market_sum.naver'
param = {'page':2} # 2페이지
response = requests.get(url, params=param)
soup = bs(response.text,'html.parser')
# soup

In [30]:
rows = soup.select('#contentarea > div.box_type_l > table.type_2 > tbody > tr')
# print(rows[0].text)
# print('-'*50)
# print(rows[1].text)
# print(len(rows))
# print(len(rows[0]))

In [31]:
title_list = []
for data in rows:
    title = data.select_one('.tltle')
    if title == None:
        pass
    else:
        title_list.append(title.text)
        # print(title.text)
title_list[:10]

['KODEX CD금리액티브(합성)',
 '한화오션',
 '삼성중공업',
 '하이브',
 '에코프로머티',
 'DB손해보험',
 '아모레퍼시픽',
 '현대로템',
 'TIGER CD금리투자KIS(합성)',
 'S-Oil']

In [32]:
# N, 종목명, 링크, code, 현재가, 시가총액, 거래량
https = 'https://finance.naver.com/'
for row in rows:
    tags = row.select('td') # td(칸)
    if len(tags) == 13:
        print(f'N : {tags[0].text}', end = ' ') # end = ' ' 엔터를 공백으로 대체
        print(f'종목명 : {tags[1].text}', end = ' ')
        print(f'링크 : {https}{tags[1].select_one("a").get("href")}', end = ' ')
        print(f'code : {tags[1].select_one("a").get("href")[-6:]}', end = ' ')
        print(f'현재가 : {tags[2].text}', end = ' ')
        print(f'시가총액 : {tags[6].text}', end = ' ')
        print(f'거래량 : {tags[9].text}', end = ' ')
        print()
        print()

N : 51 종목명 : KODEX CD금리액티브(합성) 링크 : https://finance.naver.com//item/main.naver?code=459580 code : 459580 현재가 : 1,052,575 시가총액 : 87,541 거래량 : 409,532 

N : 52 종목명 : 한화오션 링크 : https://finance.naver.com//item/main.naver?code=042660 code : 042660 현재가 : 27,550 시가총액 : 84,417 거래량 : 1,551,666 

N : 53 종목명 : 삼성중공업 링크 : https://finance.naver.com//item/main.naver?code=010140 code : 010140 현재가 : 9,590 시가총액 : 84,392 거래량 : 2,760,864 

N : 54 종목명 : 하이브 링크 : https://finance.naver.com//item/main.naver?code=352820 code : 352820 현재가 : 192,300 시가총액 : 80,097 거래량 : 106,411 

N : 55 종목명 : 에코프로머티 링크 : https://finance.naver.com//item/main.naver?code=450080 code : 450080 현재가 : 114,800 시가총액 : 79,293 거래량 : 358,034 

N : 56 종목명 : DB손해보험 링크 : https://finance.naver.com//item/main.naver?code=005830 code : 005830 현재가 : 111,000 시가총액 : 78,588 거래량 : 137,722 

N : 57 종목명 : 아모레퍼시픽 링크 : https://finance.naver.com//item/main.naver?code=090430 code : 090430 현재가 : 120,200 시가총액 : 70,308 거래량 : 186,074 

N : 58 종목명 : 현대로템 링크 : htt

### 전체 페이지 가져오기
- 가지고 온 데이터를 딕셔너리로 저장

In [33]:
dict_finance = {}
page = 1
while True:
    url = 'https://finance.naver.com/sise/sise_market_sum.naver'
    param = {'page':page} 
    response = requests.get(url, params=param)
    soup = bs(response.text,'html.parser')
    rows = soup.select('#contentarea > div.box_type_l > table.type_2 > tbody > tr')
    # 종료 조건
    if len(rows) == 1 and rows[0].text == '':
        break
    https = 'https://finance.naver.com/'
    for row in rows:
        tags = row.select('td') # td(칸)
        if len(tags) == 13:
            dict_finance[tags[0].text] = {
                'N': f'{tags[0].text}',
                '종목명' : f'{tags[1].text}',
                '링크' : f'{https}{tags[1].select_one("a").get("href")}', 
                'code' : f'{tags[1].select_one("a").get("href")[-6:]}',
                '현재가' : f'{tags[2].text}',
                '시가총액' :f'{tags[6].text}',
                '거래량' : f'{tags[9].text}'
            }
    page += 1

In [34]:
print(dict_finance['2241'])

{'N': '2241', '종목명': 'TIGER 200 산업재', '링크': 'https://finance.naver.com//item/main.naver?code=227550', 'code': '227550', '현재가': '7,330', '시가총액': '29', '거래량': '21'}


In [35]:
print(dict_finance['1'])

{'N': '1', '종목명': '삼성전자', '링크': 'https://finance.naver.com//item/main.naver?code=005930', 'code': '005930', '현재가': '59,100', '시가총액': '3,528,141', '거래량': '19,177,070'}


In [36]:
print(dict_finance['2'])

{'N': '2', '종목명': 'SK하이닉스', '링크': 'https://finance.naver.com//item/main.naver?code=000660', 'code': '000660', '현재가': '195,000', '시가총액': '1,419,605', '거래량': '3,439,726'}


In [37]:
print(dict_finance)

{'1': {'N': '1', '종목명': '삼성전자', '링크': 'https://finance.naver.com//item/main.naver?code=005930', 'code': '005930', '현재가': '59,100', '시가총액': '3,528,141', '거래량': '19,177,070'}, '2': {'N': '2', '종목명': 'SK하이닉스', '링크': 'https://finance.naver.com//item/main.naver?code=000660', 'code': '000660', '현재가': '195,000', '시가총액': '1,419,605', '거래량': '3,439,726'}, '3': {'N': '3', '종목명': 'LG에너지솔루션', '링크': 'https://finance.naver.com//item/main.naver?code=373220', 'code': '373220', '현재가': '411,500', '시가총액': '962,910', '거래량': '171,070'}, '4': {'N': '4', '종목명': '삼성바이오로직스', '링크': 'https://finance.naver.com//item/main.naver?code=207940', 'code': '207940', '현재가': '1,031,000', '시가총액': '733,804', '거래량': '64,569'}, '5': {'N': '5', '종목명': '현대차', '링크': 'https://finance.naver.com//item/main.naver?code=005380', 'code': '005380', '현재가': '220,500', '시가총액': '461,763', '거래량': '542,019'}, '6': {'N': '6', '종목명': '셀트리온', '링크': 'https://finance.naver.com//item/main.naver?code=068270', 'code': '068270', '현재가': '187,700', '시가총액

### 판다스로 내보내기

In [38]:
df = pd.DataFrame(dict_finance)
df = df.T # row, column 반전
df.head(20)

,N,종목명,링크,code,현재가,시가총액,거래량
1,1,삼성전자,https://finance.naver.com//item/main.naver?cod...,005930,"59,100","3,528,141","19,177,070"
2,2,SK하이닉스,https://finance.naver.com//item/main.naver?cod...,000660,"195,000","1,419,605","3,439,726"
3,3,LG에너지솔루션,https://finance.naver.com//item/main.naver?cod...,373220,"411,500","962,910","171,070"
4,4,삼성바이오로직스,https://finance.naver.com//item/main.naver?cod...,207940,"1,031,000","733,804","64,569"
5,5,현대차,https://finance.naver.com//item/main.naver?cod...,005380,"220,500","461,763","542,019"
6,6,셀트리온,https://finance.naver.com//item/main.naver?cod...,068270,"187,700","407,349","301,563"
7,7,삼성전자우,https://finance.naver.com//item/main.naver?cod...,005935,"48,650","400,334","1,438,175"
8,8,기아,https://finance.naver.com//item/main.naver?cod...,000270,"94,100","376,267","998,639"
9,9,KB금융,https://finance.naver.com//item/main.naver?cod...,105560,"93,900","369,523","1,423,751"
10,10,POSCO홀딩스,https://finance.naver.com//item/main.naver?cod...,005490,"340,000","280,923","228,581"


In [23]:
df.to_csv(out_path+'/naver_finance.csv')

### 엑셀로 내보내기

In [24]:
# 엑셀 파일 안 시트를 오늘 날짜로 설정
now = datetime.datetime.now() # 현재 날짜와 시간
date = f'{now.year}-{now.month}-{now.day}'
date

'2024-10-29'

In [25]:
if not os.path.exists(out_path+'/naver_finance.xlsx'):
    with pd.ExcelWriter(out_path+'/naver_finance.xlsx', mode = 'w', engine='openpyxl') as file:
        df.to_excel(file, index = False, sheet_name = date)
else:
    with pd.ExcelWriter(out_path+'/naver_finance.xlsx', mode = 'a', engine='openpyxl') as file:
        df.to_excel(file, index = False, sheet_name = date)

## 2 CGV -> 무비차트 
- 순위별로 영화 제목, 영화 포스터, 개봉일, 예매율 저장

In [39]:
url = 'http://www.cgv.co.kr/movies/'
response = requests.get(url)
soup = bs(response.text,'html.parser')

In [40]:
# 이미지, box content 포함
box_li = soup.select('#contents > div.wrap-movie-chart > div.sect-movie-chart > ol > li')
# box_li

In [43]:
now = datetime.datetime.now()
path_save =out_path+'/cgv_movie_chart/'
date = f'{now.year}-{now.month}-{now.day}_chart/'

if not os.path.exists(path_save):
    os.makedirs(path_save)
if not os.path.exists(path_save+date):
    os.makedirs(path_save+date)

In [44]:
https = 'http://www.cgv.co.kr/'
for idx, content in enumerate(box_li,start=1):
    # 제목
    title = (content.select_one('.title').text).replace("?","")
    # 개봉일
    open_date = content.select_one('.txt-info').text[:45].strip()
    # 예매율
    ticket = content.select_one('.percent').text[3:]
    # 이미지
    img_tag = content.select_one('#contents > div.wrap-movie-chart > div.sect-movie-chart > ol > li > div.box-image > a > span > img')
    movie = img_tag.get('src')
    # 링크
    link = https + content.select_one('a').get('href')
    print(f'{str(idx)+"위":=^40}')
    print(f'title : {title}')
    print(f'개봉일: {open_date}')
    print(f'예매율 : {ticket}')
    print(f'링크 : {link}')
    # 이미지 파일 저장
    response_movie = requests.get(movie)
    file_name = f'{path_save+date}'   
    with open(f'{file_name}{idx}위_{title}.jpg','wb') as file:
        file.write(response_movie.content)
# 텍스트 파일 저장
with open(f'{file_name}movie-chart.text','w') as file:
    for idx, content in enumerate(box_li,start=1):
        # 제목
        title = content.select_one('.title').text
        # 개봉일
        open_date = content.select_one('.txt-info').text[:45].strip()
        # 예매율
        ticket = content.select_one('.percent').text[3:]
        # 이미지
        img_tag = content.select_one('#contents > div.wrap-movie-chart > div.sect-movie-chart > ol > li > div.box-image > a > span > img')
        movie = img_tag.get('src')
        # 링크
        link = https + content.select_one('a').get('href')

        file.write(f'{str(idx)+"위":=^40}\n')
        file.write(f'title : {title}\n')
        file.write(f'개봉일: {open_date}\n')
        file.write(f'예매율 : {ticket}\n')
        file.write(f'링크 : {link}\n\n')

===================1위===================
title : 베놈: 라스트 댄스
개봉일: 2024.10.23
예매율 : 12.4%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88526
===================2위===================
title : 리틀 엠마
개봉일: 2024.10.25
예매율 : 3.2%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88813
===================3위===================
title : 보통의 가족
개봉일: 2024.10.16
예매율 : 2.9%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88688
===================4위===================
title : 대도시의 사랑법
개봉일: 2024.10.01
예매율 : 2.6%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88538
===================5위===================
title : 오후 네시
개봉일: 2024.10.23
예매율 : 2.5%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88804
===================6위===================
title : 임영웅ㅣ아임 히어로 더 스타디움
개봉일: 2024.08.28
예매율 : 1.8%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88499
===================7위===================
title : 너의 색
개봉일: 2024.10.12
예매율 : 1.6%
링크 : http://www.cgv.co.kr//movies/detail-view/?midx=88740

### csv, xlsx로 내보내기

In [41]:
dict_movie = {}
for idx, content in enumerate(box_li, start = 1):
    # 제목
        title = content.select_one('.title').text
        # 개봉일
        open_date = content.select_one('.txt-info').text[:45].strip()
        # 예매율
        ticket = content.select_one('.percent').text[3:]
        # 이미지
        img_tag = content.select_one('#contents > div.wrap-movie-chart > div.sect-movie-chart > ol > li > div.box-image > a > span > img')
        movie = img_tag.get('src')
        # 링크
        link = https + content.select_one('a').get('href')
        dict_movie[idx] = {
        '순위' : idx,
        'title' : title,
        '개봉일' : open_date,
        '예매율' : ticket,
        '링크' : link 
    }

In [42]:
# dict_movie

In [43]:
df = pd.DataFrame(dict_movie)
df = df.T
df.head(10)

,순위,title,개봉일,예매율,링크
1,1,아마존 활명수,2024.10.30,19.4%,https://finance.naver.com//movies/detail-view/...
2,2,베놈: 라스트 댄스,2024.10.23,13.2%,https://finance.naver.com//movies/detail-view/...
3,3,롱레그스,2024.10.30,6.8%,https://finance.naver.com//movies/detail-view/...
4,4,날씨의 아이,2024.10.30,5.9%,https://finance.naver.com//movies/detail-view/...
5,5,리틀 엠마,2024.10.25,2.8%,https://finance.naver.com//movies/detail-view/...
6,6,보통의 가족,2024.10.16,2.5%,https://finance.naver.com//movies/detail-view/...
7,7,쏜애플 불구경 2024 라이브 콘서트 필름,2024.10.30,2.2%,https://finance.naver.com//movies/detail-view/...
8,8,최소한의 선의,2024.10.30,2.0%,https://finance.naver.com//movies/detail-view/...
9,9,대도시의 사랑법,2024.10.01,2.0%,https://finance.naver.com//movies/detail-view/...
10,10,임영웅ㅣ아임 히어로 더 스타디움,2024.08.28,1.8%,https://finance.naver.com//movies/detail-view/...


In [46]:
print(path_save+'cgv_movie.xlsx')

./output/cgv_movie_chart/cgv_movie.xlsx


In [47]:
now = datetime.datetime.now() 
date = f'{now.year}-{now.month}-{now.day}'
if not os.path.exists(path_save+'cgv_movie.xlsx'):
    with pd.ExcelWriter(path_save+'cgv_movie.xlsx', mode = 'w', engine='openpyxl') as file:
        df.to_excel(file, index = False, sheet_name = date)
else:
    with pd.ExcelWriter(path_save+'cgv_movie.xlsx', mode = 'a', engine='openpyxl') as file:
        df.to_excel(file, index = False, sheet_name = date)